## <font color='red'>*********************** Molpro *****************</font>


#### MolPro is a comprehensive python package for small molecule generation using protein active site or/and similar molecules using 3D information of molecules with in-silico validation of molecules by docking , pharmacophore hypothesis. Also off target prediction based on the binding site similarities.

#### In Molpro we have diffrent diffrent features : 

### <font color='green'>1. Affinity Prediction </font>


The worldwide increase and proliferation of drug resistant microbes, coupled with the lag in new drug development, represents a major threat to human health. In order to reduce the time and cost for exploring the chemical search space, drug discovery increasingly relies on computational biology approaches. One key step in these approaches is the need for the rapid and accurate prediction of the binding affinity for potential leads. Here, we present an ensemble of three-dimensional (3D) Convolutional Neural Networks (CNNs), which combines voxelized molecular descriptors for predicting the absolute binding affinity of protein–ligand complexes. Reference : https://doi.org/10.1021/acs.jcim.0c00075

######   Preparing dataset:

We will be using a subset of PDBBind dataset given in the sample data folder for training.



Sample Data Link : https://drive.google.com/drive/folders/15x-gLYOGfXYpGjVafmNLIv33e1pm7dUY?usp=sharing

you can prepare the data for training from the raw data by running this command :

In [ ]:
"""

python data.py --data_path {path where pdb and mol2 files are stored} --hdf_path {path where processed dataset is set to be stored}
       --df_path {path to csv file containing pkd values and pdb ids} 
       
"""

###### Training model:

Once you have the dataset you can start training the model. For that can execute model.py file with the following command :

In [ ]:
"""
python model.py --hdf_path {path where dataset is stored} --train_ids_path {path where list of train ids is stored} --val_ids_path {path where list of validation ids is stored} 
                  --test_ids_path {path where list of test ids is stored}
                  --batch_size {batch size for model training} --max_epochs {epochs to train for}
                  --num_workers {number of workers for dataloader} 
                  --gpus {num_of_gpus_for_training: None for 'cpu'}
"""

after executing you will get a new folder called "lightning_logs".

###### Pkd value prediction: 

After training the model the checkpoint file saved in lightning_logs can be used for predicting the affinity of protein ligand complex. Make sure that the ligand is docked before giving to the model as input. The protein file should be protonated and not contain heteroatoms (water or ligand).

In [ ]:
from molpro.affinity_pred.predict import predict_affinity
pkd = predict_affinity(protein_file_path, protein_file_type, ligand_file_path, ligand_file_type, model_checkpoint_path)

# Input parameters :
"""
protein_file_path : str
               Path to protein file
protein_file_type : str
               File format of protein (mol2,pdb,pdbqt)
ligand_file_path : str
               Path to ligand file
ligand_file_type : str
               File format of ligand (mol2,pdb,pdbqt)
model_checkpoint_path : str 
               Path to the checkpoint of saved model
"""

In [ ]:
# returns 
"""

pkd value of the protein ligand complex : float 

"""

### <font color='green'>2. Docking </font>


AutoDock Vina is one of the fastest and most widely used open-source docking engines. It is a turnkey computational docking program that is based on a simple scoring function and rapid gradient-optimization conformational search. It was originally designed and implemented by Dr. Oleg Trott in the Molecular Graphics Lab, and it is now being maintained and develop by the Forli Lab at The Scripps Research Institute.

In this, we have created an End-to-End docking pipline, starting from docking, Protein-ligands complex generation and prediction of their interaction , then rescoring or Pkd prediction.



#### How to use?

##### <font color='blue'>1. Perform Docking </font>


In [ ]:
from molpro.docking import dock_score

binding_score=dock_score(docking_engine = None,protein = None,smile = None,out_path = None,split_out_file = None,grid_size=[40,40,40],center = [0,0,0],exhaustiveness = 32,n_poses = 10)


# Input Parameters 

"""
docking_engine: Str
            Docking Engine to be used** vina or vinardo**
protein: Str
        pdbqt protein file which need to be docked
smile: Str
        Smile which need to be docked
out_path: Str
        folder where the docked PDQT files to be saved
split_out_file: boolean
        if yes, splits the pdbqt file conatining given number poses into multiple files with one pose each. examples: out_pose_1.pdbqt,out_pose_2.pdbqt,out_pose_3.pdbqt,
        out_pose_4.pdbqt, ...., out_pose_n.pdbqt
        if no, no splitting happens
grid_size: list
        Box Size to be consider for rigid docking
center: list
        Box center to be consider for rigid docking
exhaustiveness: int
        flexibility of atoms to be given
n_poses: int
        number of poses to be consider for docking
"""

In [ ]:
# returns 

"""Top_energies: list
            Binding affinity scores of n_poses 
Generates PDBQT file [out.pdbqt, having the all poses informarion], if split_out_file=True, files will be geneated as mention above."""

Output: Binding affinity scores of n_poses like:..
[-6.206, -6.143, -6.136, -6.054, -6.027, -5.999, -5.923, -5.802, -5.75, -5.593]

###### <font color='blue'>2. Preparing the complex files for protein-Ligand interaction calculation </font>


In [ ]:
from molpro.docking import complex_prep
complex_prep(protein = None,ligand= None ,ligand_file_type = None)

# INput parameters
"""
protein: str
        Path to protein ,file format PDB
ligand: str
        Path to ligand file
ligand_file_type: str
        File format of ligand (mol2,pdb,pdbqt)
"""

###### <font color='blue'>3. Protein ligand Interaction calculation </font>


In [ ]:
from molpro.docking.dock import Pro_lig_int
interaction = Pro_lig_int(complex=None)


"""
complex: str
        path for the complex file, file format PDB
"""

In [ ]:
# Returns 
interaction: PLI like:..

{'hydrophobic': [{'RESNR': 212, 'RESTYPE': 'LEU', 'RESCHAIN': 'A', 'RESNR_LIG': 1, 'RESTYPE_LIG': 'UNL', 'RESCHAIN_LIG': 'Z', 'DIST': '3.96', 'LIGCARBONIDX': 1460, 'PROTCARBONIDX': 518, 'LIGCOO': (26.163, 31.117, 21.82), 'PROTCOO': (23.244, 29.905, 24.2)},
{'RESNR': 261, 'RESTYPE': 'TYR', 'RESCHAIN': 'A', 'RESNR_LIG': 1, 'RESTYPE_LIG': 'UNL', 'RESCHAIN_LIG': 'Z', 'DIST': '3.49', 'LIGCARBONIDX': 1463, 'PROTCARBONIDX': 800, 'LIGCOO': (27.579, 28.735, 20.895), 'PROTCOO': (27.125, 25.332, 21.507)}], 
'hbond': [{'RESNR': 241, 'RESTYPE': 'SER', 'RESCHAIN': 'A', 'RESNR_LIG': 1, 'RESTYPE_LIG': 'UNL', 'RESCHAIN_LIG': 'Z', 'SIDECHAIN': True, 'DIST_H-A': '3.59', 'DIST_D-A': '4.04', 'DON_ANGLE': '110.52', 'PROTISDON': True, 'DONORIDX': 649, 'DONORTYPE': 'O3', 
'ACCEPTORIDX': 1485, 'ACCEPTORTYPE': 'Nar', 'LIGCOO': (29.472, 32.75, 21.679), 'PROTCOO': (31.738, 35.553, 23.5)}, {'RESNR': 243, 'RESTYPE': 'SER', 'RESCHAIN': 'A', 'RESNR_LIG': 1, 'RESTYPE_LIG': 'UNL', 
'RESCHAIN_LIG': 'Z', 'SIDECHAIN': True, 'DIST_H-A': '3.14', 'DIST_D-A': '3.47', 'DON_ANGLE': '101.77', 'PROTISDON': True, 'DONORIDX': 663, 'DONORTYPE': 'O3', 'ACCEPTORIDX': 1486, 'ACCEPTORTYPE': 'Nar', 'LIGCOO': (29.412, 32.754, 23.0), 
'PROTCOO': (27.882, 35.066, 25.094)}, {'RESNR': 260, 'RESTYPE': 'GLU', 'RESCHAIN': 'A', 'RESNR_LIG': 1, 'RESTYPE_LIG': 'UNL', 'RESCHAIN_LIG': 'Z', 'SIDECHAIN': True, 'DIST_H-A': '3.12', 'DIST_D-A': '3.54', 'DON_ANGLE': '109.13', 'PROTISDON': True, 'DONORIDX': 794, 
'DONORTYPE': 'O3', 'ACCEPTORIDX': 1487, 'ACCEPTORTYPE': 'Nar', 'LIGCOO': (30.318, 31.87, 23.532), 'PROTCOO': (31.314, 30.247, 26.514)}]}

### <font color='green'>3. Geomol </font>


Prediction of a molecule’s 3D conformer ensemble from the molecular graph holds a key role in areas of cheminformatics and drug discovery. We are using a machine learning approach to generate distributions of low-energy molecular 3D conformers. Leveraging the power of message passing neural networks (MPNNs) to capture local and global graph information, we predict local atomic 3D structures and torsion angles, and using these we are assembling the whole conformer for that molecule. The original work is done by Department of Chemical Engineering, MIT, Cambridge. For more details visit :  https://arxiv.org/pdf/2106.07802.pdf


######  For training the model ->

###### Data for training:


GEOM is a dataset with over 37 million molecular conformations annotated by energy and statistical weight for over 450,000 molecules. Here we will use a subset of it. you can download the dataset by clicking the link bellow

https://dataverse.harvard.edu/api/access/datafile/4327252


After downloading and unzipping the file you will get a folder name rdkit_folder under that 2 diffrent dataset will be there.



In [ ]:
"""

rdkit_folder/
             drugs/
                    smile_seq1.pickle
                    smile_seq2.pickle
                    .
                    .
                    304340 pickle files
             qm9/
                    smile_name1.pickle
                    smile_name2.pickle
                    .
                    .
                    133232 pickle files

"""

or if you just want to try on sample datset then we have created a sample dataset by randomly selecting datapoints from original dataset. You can download the sample dataset through the link :

https://drive.google.com/drive/folders/1Y7gktdSyq3iu6OmHXVdUfgANvtOq9GU_?usp=sharing

###### Training of model:

Once you have the dataset you can start training the model. For that can execute model.py file with the following command :



In [ ]:
"""

python model.py --data_dir {pat_of_the_dataset_directory(drugs or qm9)}  --n_epochs {max_number_of_epoch} --
dataset {which_dataset_will_be_used_for_training_('drugs' or 'qm9')

"""

after executing you will get a new folder called "lightning_logs".



###### Generating similiar molecules with trained model ->

After training of model you can start generating 3d representation of molecules. For this you can use generate_conformers function from predict.py script. This function takes a list of smiles and returns a dictionary containing smile as the key and values as a list of generated conformers.



In [10]:
from molpro.geomol.predict import generate_conformers
generated_conformers =  generate_conformers(input_smiles:List[str],hparams_path:str=None,checkpoint_path :str=None,
                            n_conformers:int=10,dataset :str ="drugs",mmff: bool =False) 

"""
    input_smiles : List[str]
                   those simles for which you want to generated 3d-conformer. input should be in ['smile_1,smiles_2,.....,smile_n] format
    hparams_path : str
                   path for the hparams.yaml file. which is under lightning_logs folder
    ckpt_path    : str 
                   path of the ckpt file. which is under lightning_logs/checkpoints directory
    n_conformers : int
                   how many number of conformers you want to generate per smile
    dataset : str
                   By which model you want to generate the conformers ('drugs' or 'qm9')
    mmff : bool 
                   want to optimize the generated conformers by adding 'MMff94s' energy.?
                   """

In [ ]:
# returns
"""generated_conformers = {"smile_1":[gen_conf_1,gen_conf_2......,gen_conf_n],
                        "smile_2":[gen_conf_1,gen_conf_2......,gen_conf_n],
                        .
                        .
                        .
                        "smile_+str(len(input_smiles))": [gen_conf_1,gen_conf_2.......,gen_conf_n] }"""

In [ ]:
from rdkit import Chem
from ipywidgets import interact, fixed, IntSlider
import ipywidgets
import py3Dmol

In [ ]:
def show_mol(mol, view, grid):
    mb = Chem.MolToMolBlock(mol)
    view.removeAllModels(viewer=grid)
    view.addModel(mb,'sdf', viewer=grid)
    view.setStyle({'model':0},{'stick': {}}, viewer=grid)
    view.zoomTo(viewer=grid)
    return view

def view_single(mol):
    view = py3Dmol.view(width=600, height=600, linked=False, viewergrid=(1,1))
    show_mol(mol, view, grid=(0, 0))
    return view

def MolTo3DView(mol, size=(600, 600), style="stick", surface=False, opacity=0.5, confId=0):
    """Draw molecule in 3D
    
    Args:
    ----
        mol: rdMol, molecule to show
        size: tuple(int, int), canvas size
        style: str, type of drawing molecule
               style can be 'line', 'stick', 'sphere', 'carton'
        surface, bool, display SAS
        opacity, float, opacity of surface, range 0.0-1.0
    Return:
    ----
        viewer: py3Dmol.view, a class for constructing embedded 3Dmol.js views in ipython notebooks.
    """
    assert style in ('line', 'stick', 'sphere', 'carton')
    mblock = Chem.MolToMolBlock(mol[confId])
    viewer = py3Dmol.view(width=size[0], height=size[1])
    viewer.addModel(mblock, 'mol')
    viewer.setStyle({style:{}})
    if surface:
        viewer.addSurface(py3Dmol.SAS, {'opacity': opacity})
    viewer.zoomTo()
    return viewer

def conf_viewer(idx, mol):
    return MolTo3DView(mol, confId=idx).show()

In [ ]:
index_of_smile = 0 # index of the smile in input_smile list for which you want to visualise generated conformers

smile = input_smiles[index_of_smile]
print("Smile is :",smile)

mol_graph = Chem.MolFromSmiles(smi)
print("Graph of smile is :\n")
display(mol_graph)

mols = generated_conformers.get(smile)
interact(conf_viewer, idx=ipywidgets.IntSlider(min=0, max=len(mols)-1, step=1), mol=fixed(mols));

### <font color='green'>4. Shape Alignment </font>


###### <font color='green'>To align small molecules and calculate their 3D similarity between them. </font>


In drug discovery, common atomic level information of the small molecules / drugs aren't avaiable. In such cases, 3D arrangement (or superposition) of putative ligands have been utilized to conclude underlying necessities for organic movement. Various techniques are proposed for little atomic superposition or primary arrangement. These techniques can be ordered generally into two kinds, in particular point-based and property-based strategies. In point-based strategies, sets of molecules or pharmacophoric focuses are superposed by the least-squares fitting. Notwithstanding, in property-based techniques, different sub-atomic properties are used for superposition, including electron thickness, sub-atomic volume or shape, charge conveyance or sub-atomic electrostatic potential (MEP), hydrophobicity, hydrogen holding capacity, etc.

#### There are two function under this feature: 

###### <font color='blue'> 1. To find similarities of one smile with user's small size custom library </font>

###### How to use it?

In [ ]:
from molpro.shape_alignment import UPA

input_smiles=[smile1,smile2,smile3,....,smileN]

data_out=UPA(query_smile,input_smiles)

# Input Parametrs
"""
input_smiles : List[str]
          Simles for which you want to Find similarity with query smile. input should be in ['smile_1,smiles_2,.....,smile_n] format
query_smile: str
    Smile for which you wnat to find the similarities. 
"""

In [ ]:
# Returns :
"""
 data_out: pandas DataFrame
          DataFrame with smiles,taget_id(Library ID),fp_score(fingerprint score),Shape_similarity
          (3D shape similarity score),query_path(user smile's aligned SDF file path),hit_path(aligned SDF file path
          Smile from library which is similary to query smiles ) 
"""

###### <font color='blue'> 2. To screen a huge library which meets the similarity thresholds with given query smile. </font>

###### How to use it?


In [ ]:
from molpro.shape_alignment import library_screening

library_path="chembl/surechembl path"

data_out=UPA(library_path,smile)

# Input Parameters
"""
library_path : Str,
        Chembl / Surechembl or any huge libraries.Pickle File contains the ID,SMILES, and Fingerprint(
            morgan figerprint radius 2 and bits 1024) 
smile: Str,
        Query smile whose similar smiles need to be screened
thres: float
        Minimum 3D similarity score that smiles need to meet. Default: 0.6
"""

In [ ]:
# Returns 

"""data_out: pandas DataFrame
        DataFrame with smiles,taget_id(Library ID),fp_score(fingerprint score),Shape_similarity
        (3D shape similarity score),query_path(user smile's aligned SDF file path),hit_path(aligned SDF file path
        Smile from library which is similary to query smiles )"""

### <font color='green'>5. Shape Based Generator: </font>


The generative design of novel scaffolds and functional groups can cover unexplored regions of chemical space that still possess lead-like properties. Here we are using an AI approach to generate novel molecules starting from a seed compound, its three-dimensional (3D) shape. A variational autoencoder is used to generate the 3D representation of a compound, followed by a system of convolutional for encoding and recurrent neural networks that generate a sequence of SMILES tokens. The generative design of novel scaffolds and functional groups can cover unexplored regions of chemical space that still possess lead-like properties.



###### For training the model ->

###### Data for training:

 We will be using a subset of Zinc15 dataset for our model training. That will only have drug like smiles. you can download the dataset by clicking the link given bellow:

http://pub.htmd.org/zinc15_druglike_clean_canonical_max60.zip

After downloading unzipping the file you will get a .smi file as name "zinc15_druglike_clean_canonical_max60.smi". which will have smiles.



or if you just want to try on sample datset then we have created a sample dataset by randomly selecting datapoints from original dataset. You can download the sample dataset through the link :

https://drive.google.com/drive/folders/1CBakMNrUu-mJdH6oJJzNT1dcDVj6ECJV?usp=sharing

###### Training of model:


Once you have the dataset you can start training the model. For that can execute model.py file with the following command through your terminal:

In [ ]:
"""
python model.py --input_path {path_for_.smi_file} --batch_size {your_batch_size} 
                       --max_epochs {max_numnber_of_epochs}
                       --num_workers {num_of_workers}
                       --device {'cpu'_or_'gpu'} --gpus {num_of_gpus_for_training}
"""


after executing you will get a new folder called "lightning_logs" in the working directory.



###### Generating similiar molecules with trained model ->

After training of model you can start generating 3d representation of molecules. For this you can use generate_conformers function from predict.py script. This function takes a list of smiles and returns a dictionary containing smile as the key and values as a list of generated conformers.

In [ ]:
from molpro.geomol.predict import generate_conformers
generated_conformers =  generate_conformers(input_smiles:List[str],hparams_path:str=None,checkpoint_path :str=None,
                            n_conformers:int=10,dataset :str ="drugs",mmff: bool =False) 

"""
    input_smiles : List[str]
                   those simles for which you want to generated 3d-conformer. input should be in ['smile_1,smiles_2,.....,smile_n] format
    hparams_path : str
                   path for the hparams.yaml file. which is under lightning_logs folder
    ckpt_path    : str 
                   path of the ckpt file. which is under lightning_logs/checkpoints directory
    n_conformers : int
                   how many number of conformers you want to generate per smile
    dataset : str
                   By which model you want to generate the conformers ('drugs' or 'qm9')
    mmff : bool 
                   want to optimize the generated conformers by adding 'MMff94s' energy.?
                   """

In [ ]:
# returns 

"""
generated_smiles = {"smile_1":[gen_smi_1,gen_smi_2......,gen_smi_n],
                    "smile_2":[gen_smi_1,gen_smi_2......,gen_smi_n],
                    .
                    .
                    .
                    "smile_+str(len(input_smiles))": [gen_smi_1,gen_smi_2.......,gen_smi_n] }
"""

### <font color='green'>6. Site Based Gen </font>


###### <font color='green'>Generating novel molecules accoding to a given protein binding site
</font>


A novel method was developed to generate focused virtual libraries of small molecules based on the protein structure using deep learning-based generative models. Structures of protein–ligand complexes obtained from ligand docking are used to train a generative adversarial model to generate compound structures that are complementary to protein but also maintain diversity among themselves. Reference: https://doi.org/10.1021/acs.molpharmaceut.9b00634

######  Preparing dataset:

We will be using a subset of PDBBind dataset given in the sample data folder for training. you can download the sample data from given link :

https://drive.google.com/drive/folders/1pmoC4uBAiCkZHwYYaCAJg0OS3qlR-cgs?usp=sharing

you can prepare the data for training from the raw data by running this command :



In [ ]:
"""

python data.py --data_path {path where pdb and mol2 files are stored} 
                   --hdf_path {path where processed dataset is set to be stored}
                   --df_path {path to csv file containing pdb ids and associated smiles} 

"""

##### Training model: 

Once you have the dataset you can start training the model. For that can execute model.py file with the following command :

In [ ]:
"""

python model.py --hdf_path {path where dataset is stored} --train_ids_path {path where list of train ids is stored} 
            --val_ids_path {path where list of validation ids is stored} 
            --test_ids_path {path where list of test ids is stored} 
            --batch_size {batch size for model training} --max_epochs {epochs to train for} 
            --num_workers {number of workers for dataloader} --gpus {num_of_gpus_for_training: None for 'cpu'}

"""

after executing you will get a new folder called "lightning_logs".



###### Generate novel molecules: 

After training the model the checkpoint file saved in lightning_logs can be used for generating novel molecules. The protein file should be protonated and not contain heteroatoms (water or ligand).

In [ ]:
from molpro.site_based_gen.predict import generate_smiles
smiles = generate_smiles(protein_file_path, protein_file_type, generator_checkpoint_path, 
                         captioning_checkpoint_path, generator_steps, decoding_steps)

# Input Parameters
"""
protein_file_path : str
               Path to protein file
protein_file_type : str
               File format of protein (mol2,pdb,pdbqt)
generator_checkpoint_path : str
               Path to the checkpoint of saved generated model
captioning_checkpoint_path : str
               Path to the checkpoint of saved shape captioning model
generator_steps : str 
               The number of times a unique ligand shape is produced
decoding_steps : str 
               The number of times a unique ligand shape is decoded into SMILES
"""

In [ ]:
# Returns :
"""

A list of SMILES containing less than or equal to generator_steps x decoding_steps entries : List[str]

"""

### <font color='green'>7. Site Prediction </font>


###### <font color='green'>Predicting the binding site of a protein ligand complex (Binding site prediction)</font>


The aim of rational drug design is to discover new drugs faster and cheaper. Much of the effort is put into improving docking and scoring methodologies. However, most techniques assume that the exact location of binding sites – also referred to as pockets or binding cavities – is known. Such pockets can be located both on a surface of a single protein (and be used to modulate its activity) or at protein-protein interaction (PPI) interfaces (and be used to disrupt the interaction). This task is very challenging and we lack a method that would predict binding sites with high accuracy – most methods are able to detect only 30%–40% of pockets. In our case, the input is a 3D structure of a protein represented with a grid that can be analyzed in the same manner as 3D images, whereas the desired object is the binding pocket. Our model is based on U-Net – a state of the art model for image segmentation. The model takes protein structure as input, automatically converts it to a 3D grid with features, and outputs probability density – each point in the 3D space has assigned probability of being a part of a pocket. Reference : https://doi.org/10.1038/s41598-020-61860-z

###### Preparing dataset:

We will be using a subset of scPDB dataset given in the sample data folder for training. You can download the sample data from given link :


https://drive.google.com/drive/folders/1Z6WV3Pk6EQgUtWMEHn7xx1zhh2_dhFlh?usp=sharing

you can prepare the data for training from the raw data by running this command :



In [ ]:
"""

python data.py --data_path {path where pdb and mol2 files are stored} 
             --hdf_path {path where processed dataset is set to be stored}

"""

###### Training model:


Once you have the dataset you can start training the model. For that can execute model.py file with the following command :

In [ ]:
"""

python model.py --hdf_path {path where dataset is stored} --train_ids_path {path where list of train ids is stored} 
               --val_ids_path {path where list of validation ids is stored} 
               --test_ids_path {path where list of test ids is stored} 
               --batch_size {batch size for model training} --max_epochs {epochs to train for} 
               --num_workers {number of workers for dataloader} --gpus {num_of_gpus_for_training: None for 'cpu'}


"""

after executing you will get a new folder called "lightning_logs" in the same directory.

###### Binding site prediction:

In [ ]:
from molpro.site_pred.predict import predict_pocket_atoms, write_to_file

mols = predict_pocket_atoms(protein_file_path, protein_file_type, model_checkpoint_path) # Returns openbabel mol objects
write_to_file(mols, file_path, file_type)

# Input Parameters
"""
protein_file_path : str
               Path to protein file
protein_file_type : str
               File format of protein (mol2,pdb,pdbqt)
file_path : str
               Path to output file
file_type : str
               File format of output file (mol2,pdb,pdbqt)
model_checkpoint_path : str 
               Path to the checkpoint of saved model
"""

In [ ]:
# Returns :

"""
Output files are stored according to output format and file name
"""